# Pending

- Labeling revisar pagina
- hacer pagina pricipal (index.md) para processing, indicando como se pueden usar en conjunto

- tal vez despues hacer una pagina principal de entre todas, o en la misma pagina principal, indicar cual seria el uso ideal (el proceso a seguir), en caso de que no utilizaran a los
proveedores disponibles como data source y tuvieran sus propios datos.

In [ ]:
# Importing our First Pipeline
from finml_core.pipelines.data_factory import DatasetGenerator
# Importing Financial Splitting Methods
from finml_core.model_selection.split import PurgedKFold, purged_train_test_split

# External Libraries
import numpy as np

# Machine learning external modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

## Config for the Dataset Generation

In [5]:
import pandas as pd
import numpy as np
max(0, 0.00000001)

1e-08

In [4]:
100/0

ZeroDivisionError: division by zero

In [ ]:
# Configuration

# Choosing financial and market theoretical metrics that we want as our features.
features_config = {
    "log_ret": dict(period=1),
    "rsi": dict(period=14),
    "bollinger": dict(window=20, std=2.0),
    "macd": dict(fast=12, slow=26, signal=9),
    "rvol": dict(window=20)
}

# Choosing metrics an calculations that we're going to choose as our features,
# and including lag 5 for al our features
# The following are the most recomended metrics to use as featues.
feature_selection = {
    'log_ret': [],
    'rsi': [],
    'rsi_diff': [],
    'bb_pct_b': [],
    'bb_width': [],
    'macd_rel_hist': [],
    'rvol': []
}

# 5 lags of information per feature
for sel in feature_selection:
    feature_selection[sel] = [i for i in range(1, 4)]

# Setting parameters to compute the triple barrier method labeling
label_config = dict(
    stop_loss_multiplier = 1,
    take_profit_multiplier = 1,
    time_limit = 10,
    vol_span = 50
)

# Setting the desired stocks to download and the start date, as well as the
# etf reference date
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'AVGO']
start = '2015-01-01'
etf_reference = '^GSPC'

## Generating Dataset

In [ ]:
# Create Dataset generator instance
dataset_generator = DatasetGenerator(
    data_source='yfinance',
    feature_config=features_config,
    feature_selection=feature_selection,
    label_config=label_config
)

# Getting X and Y
X, y = dataset_generator.run(
    tickers=tickers,
    etf_reference=etf_reference,
    start_date=start
)

# This contains all the different calculations made by the feature and the
# labeling modules, excluding the lags
analysis_data = dataset_generator.analysis_data

--- 1. Starting Ingestion (1 tickers) ---


[*********************100%***********************]  2 of 2 completed

--- 2. Generating Features (X) ---
--- 3. Generating Labels (y) ---
--- 4. Consolidating & Cleaning ---
--- Pipeline Finished. Final Dataset: 2708 rows ---


## Split Data into Train & Test

In [ ]:
X_train, y_train, X_test, y_test = purged_train_test_split(
    X, y,
    t1=analysis_data['t1'],
    date_level='Date',
    test_size=0.2,
    pct_embargo=0.01
)

# Obtain training "end date" for each label, for PurgedKFold
training_mask = analysis_data.index.isin(X_train.index)
t1 = analysis_data.loc[training_mask, 't1']

## Purged K-Fold CV Example with Random Forest

In [ ]:
# Creating purged k-fold cross validation instance
purged_cv = PurgedKFold(
    n_splits=5,             # Number of folds
    t1=analysis_data['t1'], # labels exit dates
    date_level='Date',      # Date level index reference
    pct_embargo=0.01        # Pct. Embargo
)

# Crearting random forest instance
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=3,
    random_state=42
)

# Computing Purged K-Fold Cross Validation and Calculating Scores
scores = cross_val_score(
    estimator=rf_model,
    X=X_train,
    y=y_train,
    cv=purged_cv,
    scoring='accuracy',
    n_jobs=-1,  # Usa todos los núcleos del CPU para acabar rápido
    verbose=1   # Te mostrará una barra de progreso o logs básicos
)

print("\n--- Resultados de PurgedKFold ---")
print(f"Scores por Fold: {scores}")
print(f"Accuracy Promedio: {np.mean(scores):.2%}")
print(f"Desviación Estándar: {np.std(scores):.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.



--- Resultados de PurgedKFold ---
Scores por Fold: [0.47689464 0.43738447 0.48544362 0.41226852 0.45601852]
Accuracy Promedio: 45.36%
Desviación Estándar: 0.0266


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    5.4s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.5s finished
